In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Code

In [2]:
def get_dict_keys_from(df, column, book_id=1709):
    return (df[column]
            [book_id]
            [0]  # item index
            .keys()
           )

**Table of Contents**
* [Exploring and Modeling](#Exploring-and-Modeling)
    * [Publishers columns](#Publishers-column)
    * [Contributors column](#Contributors-column)
    * [Formats column](#Formats-column)
    * [Subjects column](#Subjects-column)
    * [Reviews column](#Reviews-column)
* [Creating...](#Creating...)
    * [... Constraints](#...-Constraints)
    * [Loading](#Loading)
    * [... Labels](#...-Labels)
    * [...Relationships](#...-Relationships)
    * [Both](#Both)
* [Notes](#Notes)
    * [Duplicated nodes](#Duplicated-nodes)
    * [Ambiguity](#Ambiguity)

# Exploring and Modeling

In [3]:
df = pd.read_json('Data/OTL_textbooks_v0.json', orient='index',)

print(df.info())
df.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1523 entries, 1709 to 18
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   title                   1523 non-null   object             
 1   edition_statement       210 non-null    object             
 2   volume                  49 non-null     object             
 3   copyright_year          1491 non-null   float64            
 4   ISBN10                  19 non-null     float64            
 5   ISBN13                  632 non-null    object             
 6   license                 1523 non-null   object             
 7   language                1523 non-null   object             
 8   description             1523 non-null   object             
 9   contributors            1523 non-null   object             
 10  subjects                1523 non-null   object             
 11  publishers              1523 non-null   object 

,title,edition_statement,volume,copyright_year,ISBN10,ISBN13,license,language,description,contributors,subjects,publishers,formats,rating,textbook_reviews_count,reviews,url,updated_at
1709,Applications of Educational Technology,None,None,2018.0,NaN,None,Attribution-NonCommercial,eng,Welcome to the Applications of Educational Tec...,"[{'id': 7214, 'contribution': 'Author', 'prima...","[{'id': 5, 'name': 'Education', 'parent_subjec...","[{'id': 1683, 'name': 'Oklahoma State Universi...","[{'id': 4357, 'format': 'Online', 'url': 'http...",NaN,0,[],https://open.umn.edu/opentextbooks/textbooks/a...,2024-08-02 04:30:37+00:00
1708,Measurement and Instrumentation: An Introducti...,1st Edition,None,2020.0,NaN,None,Attribution-NonCommercial-ShareAlike,eng,Measurement and instrumentation are fundamenta...,"[{'id': 7212, 'contribution': 'Author', 'prima...","[{'id': 13, 'name': 'Engineering & Technology'...","[{'id': 1682, 'name': 'SHAREOK', 'url': 'https...","[{'id': 4356, 'format': 'PDF', 'url': 'https:/...",NaN,0,[],https://open.umn.edu/opentextbooks/textbooks/m...,2024-08-02 04:23:03+00:00


In [6]:
df.copyright_year.dropna().astype('i')

1709    2018
1708    2020
1707    2018
1706    2020
1705    2024
        ... 
13      2011
14      2022
15      2016
16      2015
18      2010
Name: copyright_year, Length: 1491, dtype: int32

I will not use **textbook_reviews_count** and **updated_at**.

In [45]:
%%HTML
<figure>
    <img src="Images/OTL data model-v0.png" width="600" height="200">
    <figcaption>Data model ver. 0</figcaption>
</figure>

## Publishers column

In [58]:
get_dict_keys_from(df, 'publishers')

dict_keys(['id', 'name', 'url', 'year', 'created_at', 'updated_at'])

In [36]:
def get_publishers_df(df):
    return (df['publishers']
            .map(lambda x: [[c['id'], c['year'], c['name'],] for c in x])
            .explode()
            .astype('str')
            .str
            .extract(r'(?P<id>\d+)\,\s(?P<year>\d{4}|None)\W+(?P<name>.+[^\'\]])')
)

In [37]:
df_pub = get_publishers_df(df)
print(df_pub.info())
df_pub.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1524 entries, 1709 to 18
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1520 non-null   object
 1   year    1520 non-null   object
 2   name    1520 non-null   object
dtypes: object(3)
memory usage: 47.6+ KB
None


,id,year,name
1709,1683,2024,Oklahoma State University
1708,1682,2024,SHAREOK


In [39]:
(df_pub
 .describe()
)

,id,year,name
count,1520,1520,1520
unique,1520,12,399
top,1683,None,Editorial Grupo AEA
freq,1,1028,65


In [40]:
(df_pub
 .query("name == 'Editorial Grupo AEA'")
 .describe()
)

,id,year,name
count,65,65,65
unique,65,4,1
top,1679,2023,Editorial Grupo AEA
freq,1,34,65


**id**, **url** and **year** will be properties from the node publisher to the node book. **updated_at** and **created_at** I will not use.

In [37]:
%%HTML
<figure>
    <img src="Images/OTL data model-v0.1.png" width="800" height="200">
    <figcaption>Data model ver. 0.1</figcaption>
</figure>

## Contributors column

In [59]:
get_dict_keys_from(df, 'contributors')

dict_keys(['id', 'contribution', 'primary', 'corporate', 'title', 'first_name', 'middle_name', 'last_name', 'location', 'background_text'])

In [5]:
def get_contributors_df(df):
    return (df['contributors']
            .map(lambda x: [[c['id'], c['contribution'], c['primary'], c['corporate'], c['location']] for c in x])
            .explode()
            .astype('str')
            .str
            .extract(r"(?P<id>\d+)\W+(?P<contribution>\w+)\W+(?P<primary>\w+)\W+(?P<corporate>\w+)\W+(?P<location>.+[^\'\]])")
            .replace({'location': r'None'}, value=np.nan)
           )

In [6]:
df_cont = get_contributors_df(df)

print(df_cont.info())
df_cont.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 3083 entries, 1709 to 18
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            3051 non-null   object
 1   contribution  3051 non-null   object
 2   primary       3051 non-null   object
 3   corporate     3051 non-null   object
 4   location      2444 non-null   object
dtypes: object(5)
memory usage: 144.5+ KB
None


,id,contribution,primary,corporate,location
1709,7214,Author,False,False,Oklahoma State University
1708,7212,Author,False,False,University of Oklahoma


In [50]:
(df_cont
 .contribution
 .value_counts()
)

contribution
Author         2750
Editor          286
Illustrator      12
Translator        3
Name: count, dtype: int64

In [49]:
df_cont.describe()

,id,contribution,primary,corporate,location
count,3051,3051,3051,3051,2444
unique,3051,4,2,2,955
top,7214,Author,False,False,Universidad Nacional de Huancavelica
freq,1,2750,2088,3027,68


**Author**, **Editor**, **Primary** and **Corporate** will be labels, but I will leave contribution as a property too. **Location** will be a node.

In [38]:
%%HTML
<figure>
    <img src="Images/OTL data model-v0.2.png" width="800" height="200">
    <figcaption>Data model ver. 0.2</figcaption>
</figure>

## Formats column

In [5]:
get_dict_keys_from(df, 'formats')

dict_keys(['id', 'format', 'url', 'price', 'isbn'])

In [45]:
(df['formats']
 .map(lambda x: [c['id'] for c in x if c['format']=='PDF'])
 .explode()
 .describe()
)

count     1516
unique    1516
top       4358
freq         1
Name: formats, dtype: int64

The **id** and **url** will be properties in the relation to the node **format**. **isbn** and **price** I will not use. 

In [39]:
%%HTML
<figure>
    <img src="Images/OTL data model-v0.3.png" width="800" height="200">
    <figcaption>Data model ver. 0.3</figcaption>
</figure>

## Subjects column

In [61]:
get_dict_keys_from(df, 'subjects')

dict_keys(['id', 'name', 'parent_subject_id', 'call_number', 'visible_textbooks_count', 'url'])

In [54]:
(df['subjects']
 .map(lambda x: [c['id'] for c in x if c['name']=='Mathematics'])
 .explode()
 .value_counts()
)

subjects
7    140
Name: count, dtype: int64

Will be a node. I will not use **call_number** and **visible_textbooks_count**.

In [40]:
%%HTML
<figure>
    <img src="Images/OTL data model-v0.4.png" width="800" height="200">
    <figcaption>Data model ver. 0.4</figcaption>
</figure>

## Reviews column

In [64]:
get_dict_keys_from(df, 'reviews', book_id=1693)

dict_keys(['id', 'first_name', 'last_name', 'position', 'institution_name', 'comprehensiveness_rating', 'comprehensiveness_review', 'accuracy_rating', 'accuracy_review', 'relevance_rating', 'relevance_review', 'clarity_rating', 'clarity_review', 'consistency_rating', 'consistency_review', 'modularity_rating', 'modularity_review', 'organization_rating', 'organization_review', 'interface_rating', 'interface_review', 'grammatical_rating', 'grammatical_review', 'cultural_rating', 'cultural_review', 'overall_rating', 'overall_review', 'created_at', 'updated_at'])

In [14]:
def get_reviews_df(df):
    return (df['reviews']
            .map(lambda x: [[c['id'], c['institution_name'],] for c in x])
            .explode()
            .astype('str')
            .str
            .extract(r"(?P<id>\d+)\W+(?P<inst_name>.+[^\'\]])")
            .dropna(how='all')
           )

In [15]:
df_rev = get_reviews_df(df)
print(df_rev.info())
df_rev.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 6817 entries, 1693 to 18
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         6817 non-null   object
 1   inst_name  6817 non-null   object
dtypes: object(2)
memory usage: 159.8+ KB
None


,id,inst_name
1693,35192,Grace College and Seminary
1686,35181,Anderson University


In [16]:
df_rev.describe()

,id,inst_name
count,6817,6817
unique,6817,874
top,35192,Portland Community College
freq,1,205


In [28]:
df_cont[df_cont.location == df_rev.inst_name.value_counts().index[0]]

,id,contribution,primary,corporate,location
1688,7158,Author,False,False,Portland Community College
1688,7159,Author,False,False,Portland Community College
1688,7160,Author,False,False,Portland Community College
1533,6676,Author,True,False,Portland Community College
1430,6294,Author,True,False,Portland Community College
999,5418,Author,False,False,Portland Community College
944,5327,Author,False,False,Portland Community College
888,5236,Author,False,False,Portland Community College
700,4839,Author,False,False,Portland Community College
699,4838,Author,False,False,Portland Community College


This is a **review** node, not **reviewer** and there is some overlep between rev.intitute_name and contributors.location , but I will leave them as two distinct nodes. Maybe in future versions I change.

In [42]:
%%HTML
<figure>
    <img src="Images/OTL data model-v0.5.png" width="800" height="200">
    <figcaption>Data model ver. 0.5</figcaption>
</figure>

# Creating...

In [43]:
%%HTML
<figure>
    <img src="Images/OTL data model-v0.5.png" width="800" height="200">
    <figcaption>Data model ver. 0.5</figcaption>
</figure>

## ... Constraints

Uniques id for the nodes.

In [10]:
constraints1 = """
CREATE CONSTRAINT bookId FOR (b:Book)
REQUIRE b.id IS UNIQUE;

CREATE CONSTRAINT subjectId FOR (s:Subject)
REQUIRE s.id IS UNIQUE;

CREATE CONSTRAINT publisherId FOR (p:Publisher)
REQUIRE p.id IS UNIQUE;

CREATE CONSTRAINT contributorId FOR (c:Contributor)
REQUIRE c.id IS UNIQUE;

CREATE CONSTRAINT reviewId FOR (r:Review)
REQUIRE r.id IS UNIQUE
"""

Code(constraints1, language='cypher')

CREATE CONSTRAINT bookId FOR (b:Book)
REQUIRE b.id IS UNIQUE;

CREATE CONSTRAINT subjectId FOR (s:Subject)
REQUIRE s.id IS UNIQUE;

CREATE CONSTRAINT publisherId FOR (p:Publisher)
REQUIRE p.id IS UNIQUE;

CREATE CONSTRAINT contributorId FOR (c:Contributor)
REQUIRE c.id IS UNIQUE;

CREATE CONSTRAINT reviewId FOR (r:Review)
REQUIRE r.id IS UNIQUE

In [16]:
constraints2 = """
CREATE CONSTRAINT publishedId FOR ()-[p:PUBLISHED]->()
REQUIRE p.id IS UNIQUE;

CREATE CONSTRAINT has_formatID FOR ()-[r:HAS_FORMAT]->()
REQUIRE r.id IS UNIQUE
"""
Code(constraints2, language='cypher')

CREATE CONSTRAINT publishedId FOR ()-[p:PUBLISHED]->()
REQUIRE p.id IS UNIQUE;

CREATE CONSTRAINT has_formatID FOR ()-[r:HAS_FORMAT]->()
REQUIRE r.id IS UNIQUE

Uniques name for the nodes.

In [17]:
constraints3 = """
CREATE CONSTRAINT formatName FOR (f:Format)
REQUIRE f.name IS UNIQUE;

CREATE CONSTRAINT languageName FOR (lg:Language)
REQUIRE lg.name IS UNIQUE;

CREATE CONSTRAINT licenseName FOR (lcs:License)
REQUIRE lcs.name IS UNIQUE;

CREATE CONSTRAINT locationName FOR (loc:Location)
REQUIRE loc.name IS UNIQUE;

CREATE CONSTRAINT institutionName FOR (i:Institution)
REQUIRE i.name IS UNIQUE
"""
Code(constraints3, language='cypher')

CREATE CONSTRAINT formatName FOR (f:Format)
REQUIRE f.name IS UNIQUE;

CREATE CONSTRAINT languageName FOR (lg:Language)
REQUIRE lg.name IS UNIQUE;

CREATE CONSTRAINT licenseName FOR (lcs:License)
REQUIRE lcs.name IS UNIQUE;

CREATE CONSTRAINT locationName FOR (loc:Location)
REQUIRE loc.name IS UNIQUE;

CREATE CONSTRAINT institutionName FOR (i:Institution)
REQUIRE i.name IS UNIQUE

## Loading

In [25]:
loading = """
:auto  // only needed when using Neo4j Browser
CALL apoc.load.json('https://github.com/V-O-A/Open-textbooks-graph-database/raw/master/Data/OTL_textbooks_v0.json') YIELD value
UNWIND keys(value) as k
CALL {
    WITH k, value
    MERGE (nb:Book {id: k})
    SET nb += {title: value[k].title, edition_statement: value[k].edition_statement, volume: value[k].volume, copyright_year: toInteger(value[k].copyright_year), ISBN10: value[k].ISBN10, ISBN13: value[k].ISBN13, description: value[k].description, rating: value[k].rating, url: value[k].url}
    MERGE (lang:Language {name: value[k].language})
    MERGE (nb)-[:AVALIABLE_IN]->(lang)
    MERGE (lcs:License {name: value[k].license})
    MERGE (nb)-[:HAS_LICENSE]->(lcs)
    FOREACH (f IN value[k].formats |
        MERGE (nfm:Format {name: f.format})
        MERGE (nfm)<-[:HAS_FORMAT {id: f.id, url: f.url}]-(nb)
        )
    FOREACH (p IN value[k].publishers |
        MERGE (npub:Publisher {name: p.name})
        MERGE (npub)-[rpub:PUBLISHED {id: p.id}]->(nb)
        SET rpub += {year: p.year, url: p.url}
    )
    FOREACH (s IN value[k].subjects |
        MERGE (nsub:Subject {id: s.id})
        SET nsub += {name: s.name, url: s.url, parent_id: s.parent_subject_id}
        MERGE (nb)-[:HAS_SUBJECT]->(nsub)
        )
    FOREACH (c IN value[k].contributors |
        MERGE (ncont:Contributor {id: c.id})
        SET ncont += {title: c.title, first_name: c.first_name, middle_name: c.middle_name, last_name: c.last_name, background: c.background_text, contribution: c.contribution, primary: c.primary, corporate: c.corporate, location: c.location}
        MERGE (ncont)-[:CONTRIBUTED]->(nb)
        )
    FOREACH (r IN value[k].reviews |
        MERGE (nrev:Review {id: r.id})
        SET nrev += {first_name: r.first_name, last_name: r.last_name, position: r.position,
        comprehensiveness_rating: r.comprehensiveness_rating, comprehensiveness_review: r.comprehensiveness_review,
        accuracy_rating: r.accuracy_rating, accuracy_review: r.accuracy_review,
        relevance_rating: r.relevance_rating, relevance_review: r.relevance_review,
        clarity_rating: r.clarity_rating, clarity_review: r.clarity_review,
        consistency_rating: r.consistency_rating, consistency_review: r.consistency_review,
        modularity_rating: r.modularity_rating, modularity_review: r.modularity_review,
        organization_rating: r.organization_rating, organization_review: r.organization_review,
        interface_rating: r.interface_rating, interface_review: r.interface_review,
        grammatical_rating: r.grammatical_rating, grammatical_review: r.grammatical_review,
        cultural_rating: r.cultural_rating, cultural_review: r.cultural_review,
        overall_rating: r.overall_rating, overall_review: r.overall_review,
        created_at: r.created_at, updated_at: r.updated_at}
        MERGE (inst:Institution {name: r.institution_name})
        MERGE (nrev)-[:IN_INSTITUTTION]->(inst)
        MERGE (nb)-[:HAS_REVIEW]->(nrev)
        )
} IN TRANSACTIONS
    ON ERROR FAIL
"""
Code(loading, language='cypher')

:auto  // only needed when using Neo4j Browser
CALL apoc.load.json('https://github.com/V-O-A/Open-textbooks-graph-database/raw/master/Data/OTL_textbooks_v0.json') YIELD value
UNWIND keys(value) as k
CALL {
    WITH k, value
    MERGE (nb:Book {id: k})
    SET nb += {title: value[k].title, edition_statement: value[k].edition_statement, volume: value[k].volume, copyright_year: toInteger(value[k].copyright_year), ISBN10: value[k].ISBN10, ISBN13: value[k].ISBN13, description: value[k].description, rating: value[k].rating, url: value[k].url}
    MERGE (lang:Language {name: value[k].language})
    MERGE (nb)-[:AVALIABLE_IN]->(lang)
    MERGE (lcs:License {name: value[k].license})
    MERGE (nb)-[:HAS_LICENSE]->(lcs)
    FOREACH (f IN value[k].formats |
        MERGE (nfm:Format {name: f.format})
        MERGE (nfm)<-[:HAS_FORMAT {id: f.id, url: f.url}]-(nb)
        )
    FOREACH (p IN value[k].publishers |
        MERGE (npub:Publisher {name: p.name})
        MERGE (npub)-[rpub:PUBLISHED {id: p.id}]->(nb)
        SET rpub += {year: p.year, url: p.url}
    )
    FOREACH (s IN value[k].subjects |
        MERGE (nsub:Subject {id: s.id})
        SET nsub += {name: s.name, url: s.url, parent_id: s.parent_subject_id}
        MERGE (nb)-[:HAS_SUBJECT]->(nsub)
        )
    FOREACH (c IN value[k].contributors |
        MERGE (ncont:Contributor {id: c.id})
        SET ncont += {title: c.title, first_name: c.first_name, middle_name: c.middle_name, last_name: c.last_name, background: c.background_text, contribution: c.contribution, primary: c.primary, corporate: c.corporate, location: c.location}
        MERGE (ncont)-[:CONTRIBUTED]->(nb)
        )
    FOREACH (r IN value[k].reviews |
        MERGE (nrev:Review {id: r.id})
        SET nrev += {first_name: r.first_name, last_name: r.last_name, position: r.position,
        comprehensiveness_rating: r.comprehensiveness_rating, comprehensiveness_review: r.comprehensiveness_review,
        accuracy_rating: r.accuracy_rating, accuracy_review: r.accuracy_review,
        relevance_rating: r.relevance_rating, relevance_review: r.relevance_review,
        clarity_rating: r.clarity_rating, clarity_review: r.clarity_review,
        consistency_rating: r.consistency_rating, consistency_review: r.consistency_review,
        modularity_rating: r.modularity_rating, modularity_review: r.modularity_review,
        organization_rating: r.organization_rating, organization_review: r.organization_review,
        interface_rating: r.interface_rating, interface_review: r.interface_review,
        grammatical_rating: r.grammatical_rating, grammatical_review: r.grammatical_review,
        cultural_rating: r.cultural_rating, cultural_review: r.cultural_review,
        overall_rating: r.overall_rating, overall_review: r.overall_review,
        created_at: r.created_at, updated_at: r.updated_at}
        MERGE (inst:Institution {name: r.institution_name})
        MERGE (nrev)-[:IN_INSTITUTTION]->(inst)
        MERGE (nb)-[:HAS_REVIEW]->(nrev)
        )
} IN TRANSACTIONS
    ON ERROR FAIL

Almost done.

## ... Labels

Contributor node

In [26]:
labels1 = """
MATCH (c:Contributor)
WHERE c.contribution = 'Author' SET c:Author;
MATCH (cont:Contributor)
WHERE cont.contribution = 'Editor' SET cont:Editor;
"""
Code(labels1, language='cypher')

MATCH (c:Contributor)
WHERE c.contribution = 'Author' SET c:Author;
MATCH (cont:Contributor)
WHERE cont.contribution = 'Editor' SET cont:Editor;

In [27]:
labels2 = """
MATCH (c:Contributor)
WHERE c.primary = TRUE
SET c:Primary;

MATCH (cn:Contributor)
WHERE cn.corporate = TRUE
SET cn:Corporate;

MATCH (cont:Contributor)
REMOVE cont.primary, cont.corporate
"""
Code(labels2, language='cypher')

MATCH (c:Contributor)
WHERE c.primary = TRUE
SET c:Primary;

MATCH (cn:Contributor)
WHERE cn.corporate = TRUE
SET cn:Corporate;

MATCH (cont:Contributor)
REMOVE cont.primary, cont.corporate

## ... Relationships

In [28]:
relationships1 = """
MATCH (cs:Subject WHERE cs.parent_id IS NOT NULL)
MATCH (ps:Subject {id: cs.parent_id})
MATCH (ps)<-[r:HAS_SUBJECT]-()
DELETE r
WITH ps, cs
    MERGE (cs)<-[:HAS_CHILD]-(ps)
    REMOVE cs.parent_id
"""
Code(relationships1, language='cypher')

MATCH (cs:Subject WHERE cs.parent_id IS NOT NULL)
MATCH (ps:Subject {id: cs.parent_id})
MATCH (ps)<-[r:HAS_SUBJECT]-()
DELETE r
WITH ps, cs
    MERGE (cs)<-[:HAS_CHILD]-(ps)
    REMOVE cs.parent_id

## Both

In [29]:
both = """
MATCH (c:Contributor WHERE c.location IS NOT NULL)
MERGE (l:Location {name: c.location})
MERGE (c)-[:IN_LOCATION]->(l)
REMOVE c.location
"""
Code(both, language='cypher')

MATCH (c:Contributor WHERE c.location IS NOT NULL)
MERGE (l:Location {name: c.location})
MERGE (c)-[:IN_LOCATION]->(l)
REMOVE c.location

# Notes

## Duplicated nodes

In [30]:
notes1 = """
MATCH (gb:Contributor {first_name: 'Gilbert', last_name: 'Strang'})-[cont:CONTRIBUTED]-(b)
MATCH (gb)-[:IN_LOCATION]-(l)
RETURN *
"""
Code(notes1, language='cypher')

MATCH (gb:Contributor {first_name: 'Gilbert', last_name: 'Strang'})-[cont:CONTRIBUTED]-(b)
MATCH (gb)-[:IN_LOCATION]-(l)
RETURN *

In [31]:
notes2 = """
MATCH ()<-[:CONTRIBUTED]-(c:Contributor)-[:CONTRIBUTED]->()
RETURN c    // will return 'none', this path does not exist
"""
Code(notes2, language='cypher')

MATCH ()<-[:CONTRIBUTED]-(c:Contributor)-[:CONTRIBUTED]->()
RETURN c    // will return 'none', this path does not exist

**FOR EACH CONTRIBUTOR, *ID* ACTUALLY INDICATES THE CONTRIBUTION TO THE BOOK, NOT ONE PERSON.**

In [32]:
notes3 = """
MATCH (l:Location)
WHERE l.name IN ['MIT', 'Massachusetts Institute of Technology']
RETURN l.name  // will return 2 nodes
"""
Code(notes3, language='cypher')

MATCH (l:Location)
WHERE l.name IN ['MIT', 'Massachusetts Institute of Technology']
RETURN l.name  // will return 2 nodes

In [33]:
notes4 = """
MATCH (l:Location)
WHERE size(l.name) <= 10
RETURN l.name
"""
Code(notes4, language='cypher')

MATCH (l:Location)
WHERE size(l.name) <= 10
RETURN l.name

SAME LOCATION STORED WITH DIFFERENT NAMES

## Ambiguity

In [35]:
notes5 = """
MATCH (l:Location)
WHERE l.name CONTAINS ','
RETURN l.name  // 'Berlin, Germany', ...
"""

Code(notes5, language='cypher')

MATCH (l:Location)
WHERE l.name CONTAINS ','
RETURN l.name  // 'Berlin, Germany', ...

STATES AND INTITUTIONS STORED WITH THE SAME LABEL